Copyright 2017 Novo Nordisk Foundation Center for Biosustainability,
Technical University of Denmark.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import sys
import warnings

In [2]:
from os.path import dirname
from importlib import import_module

In [3]:
from types import (ModuleType, FunctionType, LambdaType)
try:
    from types import (ClassType, TypeType)
    _types = (ClassType, TypeType, type)
except ImportError:
    _types = type

In [4]:
import networkx as nx
import pygraphviz as pgv

In [5]:
from setuptools import find_packages

In [6]:
from IPython.display import display

In [7]:
pkg_name = "cameo"

In [ ]:
def graph_package_hierarchy(pkg_name, graph, skip_methods=True):
    def make_id(parent, name):
        return "{}_{}".format(parent, name)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        pkg = import_module(pkg_name)

    sub_names = find_packages(dirname(pkg.__file__))
    
    tree = graph.add_subgraph(name="cluster_{}".format(pkg_name), label=pkg_name, color="black")
    mod_attr = dict(shape="folder", style="filled", fillcolor="lightgrey")
    class_attr = dict(shape="note")
    func_attr = dict(shape="plaintext")
    
    tree.add_node(pkg_name, label=pkg_name, **mod_attr)
    for mod_name in sub_names:
        abs_name = "{}.{}".format(pkg_name, mod_name)
        __import__(abs_name)
        module = sys.modules[abs_name]
        print(module.__name__)
        parent = pkg_name
        for name in mod_name.split("."):
            mod_id = make_id(parent, name)
            tree.add_node(mod_id, label=name, **mod_attr)
            tree.add_edge(parent, mod_id)
            parent = mod_id
        for name in dir(module):
            if name.startswith("_"):
                continue
            elem_id = make_id(parent, name)
            elem = getattr(module, name)
            if isinstance(elem, ModuleType):
                tree.add_node(elem_id, label=name, **mod_attr)
                tree.add_edge(parent, elem_id)
            if isinstance(elem, _types):
                tree.add_node(elem_id, label=name, **class_attr)
                tree.add_edge(parent, elem_id)
                if skip_methods:
                    continue
                for n in dir(elem):
                    if n.startswith("_"):
                        continue
                    el = getattr(elem, n)
                    print(n, type(el))
                    if not isinstance(el, (MethodType, FunctionType, property)):
                        continue
                    el_id = make_id(elem_id, n)
                    tree.add_node(el_id, label=n, **func_attr)
                    tree.add_edge(elem_id, el_id)
            elif isinstance(elem, (FunctionType, LambdaType)):
                tree.add_node(elem_id, label=name, **func_attr)
                tree.add_edge(parent, elem_id)
    return tree

In [ ]:
merge_view = pgv.AGraph(name="Merge Overview", strict=True, directed=False, rankdir="LR")
graph_package_hierarchy("cameo", merge_view)
graph_package_hierarchy("cobra", merge_view)
merge_view.draw("cameo_cobra_merge.svg", prog="dot")
merge_view.write("cameo_cobra_merge.dot")

cameo.network_analysis
cameo.models
